In [1]:
import numpy as np
import copy
from tqdm import tqdm

In [2]:
# Load the graph problem from a .tsp.txt file
data = np.loadtxt('data/eil51.tsp.txt', usecols=[1,2])

In [3]:
# Helper function to convert the coordinates into an adjacency matrix
def coordinates_to_adjacency_matrix(data):
    a = np.zeros((len(data),len(data)))
    for i in range(len(a)):
        for j in range(len(a)):
            if not i == j:
                a[i][j] = np.linalg.norm(data[i] - data[j])
    return a

In [4]:
class Chromosome():
    
    # Random generated Chromosome
    #  m - number of traveling salesmans
    def __init__(self, number_of_cities, number_of_traveling_salesman, adj = coordinates_to_adjacency_matrix(data)):
        self.n = number_of_cities
        self.m = number_of_traveling_salesman
        self.adj = adj
        c = np.array(range(1,number_of_cities))
        np.random.shuffle(c)
        self.solution = np.array_split(c, self.m)
        for i in range(len(self.solution)):
            self.solution[i] = np.insert(self.solution[i],0,0)
            self.solution[i] = np.append(self.solution[i],0)
        self.fitness()
            
    # Evaluate the Chromosome - Fitness function
    #  based on 2 features: 
    #   - overall cost (cumulated from all salesman)
    #   - worst (longest) salesman cost
    #  adj - adjacency matrix
    def fitness(self):
        self.cost = 0
        longest_salesman_fitness = []
        longest_salesman_length = 0
        for i in range(self.m):
            salesman = self.solution[i]
            salesman_fitness = 0
            for j in range(len(salesman) - 1):
                salesman_fitness = salesman_fitness + self.adj[salesman[j]][salesman[j+1]]
            self.cost = self.cost + salesman_fitness
            if len(salesman) > longest_salesman_length or (len(salesman) == longest_salesman_length and salesman_fitness > self.minmax):
                longest_salesman_length = len(salesman)
                self.minmax = salesman_fitness
        self.score = self.cost + self.minmax
    
    # Mutation operator - mutates a single Traveling Salesman
    #  by swaping 2 cities
    def mutate_local(self):
        index = np.random.randint(0,self.m)
        mutant = self.solution[index]
        i,j = np.random.randint(1,len(mutant)-1), np.random.randint(1,len(mutant)-1)
        mutant[i], mutant[j] = mutant[j], mutant[i]
        old_cost = self.cost
        self.fitness()
    
    # Mutation operator - mutates 2 Traveling Salesmans
    #  by removing a city from a salesman and asigning it to the second one
    def mutate_global(self):
        for i in range(self.m):
            if len(self.solution[i]) < 3:
                print(i, self.solution[i])
        
        
        index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        while index1 == index2:
            index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        while len(self.solution[index1]) < 4:
            index1, index2 = np.random.randint(0,self.m), np.random.randint(0,self.m)
        mutant1, mutant2 = self.solution[index1], self.solution[index2]
        i,j = np.random.randint(1,len(mutant1)-1), np.random.randint(1,len(mutant2)-1)
        self.solution[index2] = np.insert(mutant2, j, mutant1[i])
        self.solution[index1] = np.delete(mutant1, i)
        old_cost = self.cost
        self.fitness()
    

In [ ]:
chromosome = Chromosome(number_of_cities = 51, number_of_traveling_salesman = 2)
for it in tqdm(range(100000)):
    # Mutate globally
    chromosome_copy = copy.deepcopy(chromosome)
    chromosome_copy.mutate_global()
    if chromosome_copy.score < chromosome.score:
        chromosome = chromosome_copy
    # Mutate locally
    chromosome_copy = copy.deepcopy(chromosome)
    chromosome_copy.mutate_local()
    if chromosome_copy.score < chromosome.score:
        chromosome = chromosome_copy

 35%|███▌      | 35454/100000 [00:04<00:08, 8008.84it/s]

In [8]:
# Print best solution
for i in range(chromosome.m):
    print(i+1, ":  ", chromosome.solution[i][0]+1, end="", sep="")
    for j in range(1,len(chromosome.solution[i])):
        print("-", chromosome.solution[i][j]+1, end="", sep="")
    print(" --- #", len(chromosome.solution[i]))
print()

# Print cost
print("Cost: ", chromosome.cost)
print("Minmax: ", chromosome.minmax)

1:  1-15-45-44-42-19-40-41-13-25-14-24-43-22-28-3-36-35-29-16-34-30-10-39-33-1 --- # 26
2:  1-27-32-2-20-21-50-9-49-37-17-4-18-47-51-46-11-38-5-12-6-48-23-7-26-31-8-1 --- # 28

Cost:  592.772850657
Minmax:  251.11883494
